In [69]:
import hublib
import hublib.use
%use octave-4.2.0
import oct2py
%load_ext oct2py.ipython
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
from decimal import Decimal
%octave pkg load optim
import unicodedata
from unicodedata import lookup as GL
import sympy as sy

The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [2]:
from hublib.ui import FileUpload

def mycb(name,val):
    #print("CB %s (length = %s bytes)" % (name, len(val)))
    global fnm
    fnm=name

f = FileUpload("Please upload Raman spectra data file (CSV)", 
               "Raman data files should be uploaded as 2 column CSV files",cb=mycb)
f

<IPython.core.display.Javascript object>

Box(children=(HTML(value='<p data-toggle="popover" title="Raman data files should be uploaded as 2 column CSV files">Please upload Raman spectra data file (CSV)</p>', layout=Layout(flex='2 1 auto')), FileWidget()), layout=Layout(border='solid 1px lightgray', display='flex', flex_flow='row', justify_content='space-between', width='auto'))

In [3]:
def errprint(code):
    errfile=pd.read_csv('errfile.txt',sep='\t',header=None)
    with errout:
        clear_output()
        print(errfile[0][code])
        errout

In [14]:
def case_lookup(index):
    casefile=pd.read_csv('Case_List.txt',sep='\t',header=None)
    c=casefile[0][index]
    return c

In [105]:
fit_but = widgets.Button(description='Do Fitting')
    
def fit_but_cb(change):
    
    with errout:
        clear_output()
    
    f.save('tmpfile')
    if fnm[-3:]=='txt':
        sp='\t'
    elif fnm[-3:]=='csv':
        sp=','
    else:
        errprint(0)
        return
    data = pd.read_csv('tmpfile',sep=sp,header=None)
    
    srow=0;
    if type(data[0][0])==str:
        srow=1
    
    W=data[0][srow:len(data)]
    W=np.array(W);W=W.astype(float)
    I_raw=data[1][srow:len(data)]
    I_raw=np.array(I_raw);I_raw=I_raw.astype(float)
    I=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)));
    
    pG=[np.max(I), 5, 1581.6] #a w b
    pGp=[np.max(I), 18, 2675]
    PG=%octave -i pG,W,I lsqcurvefit(@Single_Lorentz,pG,W,I);
    PGp=%octave -i pGp,W,I lsqcurvefit(@Single_Lorentz,pGp,W,I,[0,0,2000],[max(I),100,3000]);
    PG[1]=np.absolute(PG[1]);PGp[1]=np.absolute(PGp[1]); #FWHM sometimes returns - bc always squared
    IGfit=%octave -i PG,W Single_Lorentz(PG,W);
    IGpfit=%octave -i PGp,W Single_Lorentz(PGp,W);
    IGfit=IGfit[0];IGpfit=IGpfit[0];
    If=IGfit+IGpfit;
    
    
    pD=[np.max(I),5,1300]
    PD=%octave -i pD,W,I lsqcurvefit(@Single_Lorentz,pD,W,I,[0,0,1200],[max(I),100,1400]);
    PD[1]=np.absolute(PD[1]);
    IDfit=%octave -i PD,W Single_Lorentz(PD,W);
    IDfit=IDfit[0]
    Q=1-(PD[0]/PG[0])
    Q=Decimal(Q[0])
    
    Cdat=np.load('Cfits.npy')

    diffs_lin=[];diffs_Gp=[];
    diffs=[];
    for d in range(6):
        
        LG=Cdat[d][0];
        LGp=Cdat[d][1];
        LGfit=%octave -i LG,W Single_Lorentz(LG,W); 
        LGpfit=%octave -i LGp,W Single_Lorentz(LGp,W);
        Lf=(LGfit+LGpfit)
 
        dfGp=np.average(np.absolute(100*(PGp-LGp)/LGp))
        dfG=np.average(np.absolute(100*(PG-LG)/LG))
        df=np.average([dfGp,dfG])
        diffs.append(df)
  
    md=np.min(diffs)
    mdi=np.argmin(diffs)

    mG=Cdat[mdi][0];mGp=Cdat[mdi][1];
    mGfit=%octave -i mG,W Single_Lorentz(mG,W);
    mGpfit=%octave -i mGp,W Single_Lorentz(mGp,W);
    mf=mGfit+mGpfit;mf=mf[0]
    
    with diffsplot:
        clear_output()
        plt.plot(diffs,'kv')
        plt.plot(mdi,md,'gv')
        plt.annotate((round(Decimal(md),2)),xy=(mdi,1.2*md))
        plt.xticks(range(6),('1','2','3','4','5','Graphite'))
        plt.xlabel('# Layers')
        plt.ylabel('$\Delta$ [%]')
        plt.show()
    with datplot:
        clear_output()
        plt.plot(W,I,'b',W,If+IDfit,'r',W,mf,'g')
        plt.xlabel('$\omega$ $[cm^{-1}]$')
        plt.ylabel('$I_{norm}$ [arb]')
        plt.legend(labels=['Raw','Fit','Test'])
        plt.annotate('Q=%1.2f' %round(Q,2) ,xy=(np.min(W),0.98))
        plt.annotate('D',xy=(0.85*PD[2],1.1*PD[0]))
        plt.annotate('G',xy=(0.9*PG[2],0.95*PG[0]))
        plt.annotate('G\'',xy=(0.94*PGp[2],0.95*PGp[0]))
        plt.show()
        
    with plist:
        clear_output()
        G=GL('GREEK CAPITAL LETTER GAMMA')
        o=GL('GREEK SMALL LETTER OMEGA')
        print('G Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
              'G\' Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
              'D Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
              'Quality=%1.2f (Ratio of D to G)\n'
              'Best Case Match: %s'
              %(PG[0],G,PG[1],o,PG[2],PGp[0],G,PGp[1],o,PGp[2],PD[0],G,PD[1],o,PD[2],Q,case_lookup(mdi)))

fit_but.on_click(fit_but_cb)
fit_but

Button(description='Do Fitting', style=ButtonStyle())

In [79]:
diffsplot=widgets.Output();diffsplot

Output()

In [80]:
datplot=widgets.Output();datplot

Output()

In [81]:
errout=widgets.Output();errout

Output()

In [82]:
plist=widgets.Output();plist

Output()